In [ ]:
# Pkg.update()

In [1]:
using JuMP, Cbc
using Combinatorics, Iterators
using StatsBase
# using PaddedViews

INFO: Precompiling module StaticArrays.
INFO: Recompiling stale cache file /Users/claus/.julia/lib/v0.6/DiffResults.ji for module DiffResults.
INFO: Recompiling stale cache file /Users/claus/.julia/lib/v0.6/ForwardDiff.ji for module ForwardDiff.
INFO: Recompiling stale cache file /Users/claus/.julia/lib/v0.6/JuMP.ji for module JuMP.
INFO: Recompiling stale cache file /Users/claus/.julia/lib/v0.6/Cbc.ji for module Cbc.
INFO: Recompiling stale cache file /Users/claus/.julia/lib/v0.6/Polynomials.ji for module Polynomials.
INFO: Recompiling stale cache file /Users/claus/.julia/lib/v0.6/Combinatorics.ji for module Combinatorics.
INFO: Recompiling stale cache file /Users/claus/.julia/lib/v0.6/StatsBase.ji for module StatsBase.


## (This is a WIP)

### Introduction
In a multi-warehouse eCommerce setting, we want to optimize the allocation of orders to warehouses on a number of criteria. We may choose to optimize freight cost, delivery time, customer experience or inventory holding cost. In many eCommerce settings, minimizing the number of shipments per orders minimized freight cost while also maximizing positive customer experience, while upholding customer delivery expectations if this is clearly communicated to customers are checkout time, or soon thereafter.

We will try to use Julia to optimize a multi-SKU, multi-warehouse scenario, minimizing the number of shipments in the face of limited inventory.

(Inspired by "Order Fulfillment in Online Retailing: What Goes Where" by Ping Josephine Xu, pg. 49
https://www.researchgate.net/publication/37994759_Order_fulfillment_in_online_retailing_what_goes_where)

### Setup

$k$&nbsp;&nbsp;&nbsp;&nbsp;index for warehouses

$I$&nbsp;&nbsp;&nbsp;&nbsp;set of SKUs, where $|I| = m$ and $i$ is the index.

$N = {1, . . . , n}$&nbsp;&nbsp;&nbsp;&nbsp;a collection of all possible subsets of
the order, i.e., $C_l$, $l ∈ N$, is the $lth$ subset of the order 

$A$&nbsp;&nbsp;&nbsp;&nbsp;a $m$ by $n$ matrix such that $a_{il}$ is the number of
of SKU $i$ included in subset $C_l$

$d_i$&nbsp;&nbsp;&nbsp;&nbsp;units of SKU $i$ in the order

$u$&nbsp;&nbsp;&nbsp;&nbsp;order size, or the number of units in the order, $u = \sum_{i}d_i$

$e_n$&nbsp;&nbsp;&nbsp;&nbsp;a $n$ by $1$ vector of 1’s

$y_{lk}$ = $1$ if subset $C_l$ is shipped out of warehouse $k$; $=0$ otherwise

$s_{ik}$&nbsp;&nbsp;&nbsp;&nbsp;inventory units of SKU $i$ available at warehouse $k$


We'll start with **3** SKUs

In [2]:
I_skus = [1,2,3]
m = length(I_skus)

3

and **2** warehouses:

In [3]:
K_warehouses = [1,2]
k = length(K_warehouses)

2

Each warehouse can carry up to `max_inventory` of each SKU

In [11]:
max_inventory = 3
inventory = rand(1:max_inventory, m, k)

3×2 Array{Int64,2}:
 3  1
 3  3
 2  1

Then we make combination of SKUs up between 2 and the number of unique SKUs we have (3)

In [12]:
min_order_size = 2
max_order_size = m

3

In [13]:
sku_combos = [collect(combinations(I_skus, o)) for o in min_order_size:max_order_size];

In [14]:
sku_combos

2-element Array{Array{Array{Int64,1},1},1}:
 Array{Int64,1}[[1, 2], [1, 3], [2, 3]]
 Array{Int64,1}[[1, 2, 3]]             

We flatten this into an array of orders:

In [15]:
J_orders = []
for i in 1:length(sku_combos)
    append!(J_orders, sku_combos[i])
end

In [16]:
J_orders

4-element Array{Any,1}:
 [1, 2]   
 [1, 3]   
 [2, 3]   
 [1, 2, 3]

In [18]:
j =length(J_orders)

4

In [19]:
# wh_allocation = [.3, .7]
# orders_wh = [sample(orders, Int(round(length(orders) * wh, 0)), replace=false) for wh in wh_allocation]
# # orders_wh = [[o, o in wc ? 1 : 2] for o in orders]

In [20]:
I_j = [unique(s) for s in J_orders]

4-element Array{Array{Int64,1},1}:
 [1, 2]   
 [1, 3]   
 [2, 3]   
 [1, 2, 3]

Then we determine all possible subsets we can make out of those orders:

In [21]:
N = [collect(subsets(j)) for j in J_orders]

4-element Array{Array{Array{Int64,1},1},1}:
 Array{Int64,1}[Int64[], [1], [2], [1, 2]]                                
 Array{Int64,1}[Int64[], [1], [3], [1, 3]]                                
 Array{Int64,1}[Int64[], [2], [3], [2, 3]]                                
 Array{Int64,1}[Int64[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]

In [22]:
n = sum([length(n_j) for n_j in N])

20

In [26]:
# for n_j in N
#     for l in n_j
#         if length(l) > 0
#             @show l
#         end
#     end
# end

In [24]:
m, n, j

(3, 20, 4)

In [28]:
# A = [N[o][l] for o=1:j, l=1:4, i=1:m]

In [30]:
# maximum([size(n,1) for n in N[4]])

In [31]:
# for o=1:j
#     for i=1:m
#         for l=2:maximum([size(n,1) for n in N[o]])
#             @show o, i, l, N[o][l]
#         end
#     end
# end

$A$ is a $m$ by $n$ matrix such that $a_{il}$ is the number of SKU $i$ included in subset $C_l$

In [37]:
A = [Int(i in (N[o][l])) for l=2:4, i=1:m, o=1:j]

3×3×4 Array{Int64,3}:
[:, :, 1] =
 1  0  0
 0  1  0
 1  1  0

[:, :, 2] =
 1  0  0
 0  0  1
 1  0  1

[:, :, 3] =
 0  1  0
 0  0  1
 0  1  1

[:, :, 4] =
 1  0  0
 0  1  0
 1  1  0

In [33]:
model = Model(solver=CbcSolver())

# decision variable (binary): whether to ship subset C_l from warehouse k
@variable(model, y[1:n, 1:j, 1:k], Bin)

# # Objective: minimize number of shipments
@objective(model, Min, sum(y))

model

Minimization problem with:
 * 0 linear constraints
 * 160 variables: 160 binary
Solver is CbcMathProg

In [ ]:
@constraint(model, y[1] + y[2] >= 1)

In [ ]:
# Solve problem using MIP solver
# status = solve(model)

In [ ]:
# println("Total # of warehouses: ", getobjectivevalue(model))

# println("Build warehouses at distribution center(s):")

# [i for i=1:m if getvalue(y[i]) == 1 ]